In [1]:
import pandas as pd
import math
import os
from string import ascii_lowercase
# df = pd.read_csv('data/letters-csv/a/a_4.csv')
df = pd.read_csv('data/sample-csv/shape_1.csv')
df.shape

(47, 3)

In [2]:
x_col = df["x"]
y_col = df["y"]
t_col = df["time"]
x_max = max(x_col)
x_min = min(x_col)
y_max = max(y_col)
y_min = min(y_col)

In [3]:
# delete rows that have repeated values that are the same as the one right above. 
# for index, row in df.iterrows():
#     if row["x"] == df.iloc[index-1]["x"] and row["y"] == df.iloc[index-1]["y"]:
#         df = df.drop(labels=index, axis=0)
df.tail(5)
iter = df.iterrows()
while True:
    nextRow = next(iter)
    if nextRow == None: break
    if nextRow["x"] == next(next)["x"] and next["y"] == next(next)["y"]:
        df = df.drop(next)
df.tail(5)

TypeError: 'tuple' object is not callable

In [ ]:
for index, row in df.iterrows():
    if row["x"] == df.iloc[index-1]["x"] and row["y"] == df.iloc[index-1]["y"]:
        df = df.drop(labels=index, axis=0)

In [ ]:
def findNonRepeated(start, *args): #args are the columns name to be checked
    for i in range (start, df.shape[0], 1):
        different = True
        for arg in args:
            if df.iloc(i)[arg] != df.iloc(i-1)[arg]: continue
            else: different = False
        if different == True:
            return i
    return -1
    

In [6]:
def delta(col, ind1, ind2=None):
    if ind2 == None:
        return col[ind1] - col[ind1-1]
    return col[ind1] - col[ind2]

In [21]:
def calculateRubine(df):

    x_col = df["x"]
    y_col = df["y"]
    t_col = df["time"]
    x_max = max(x_col)
    x_min = min(x_col)
    y_max = max(y_col)
    y_min = min(y_col)

    f1 = (x_col[2] - x_col[0]) / math.sqrt((y_col[2] - y_col[0])**2 + (x_col[2] - x_col[0])**2)
    f2 = (y_col[2] - y_col[0]) / math.sqrt((y_col[2] - y_col[0])**2 + (x_col[2] - x_col[0])**2)
    f3 = math.sqrt((y_max - y_min)**2 + (x_max - x_min)**2)
    f4 = math.atan((y_max - y_min) / (x_max - x_min))
    f5 = math.sqrt((x_col[-1:] - x_col[0])**2 + (y_col[-1:] - y_col[0])**2)
    f6 = (x_col[-1:] - x_col[0]) / f5
    f7 = (y_col[-1:] - y_col[0]) / f5

    f8 = 0
    for i in range(1, df.shape[0], 1):
        f8 += math.sqrt(delta(x_col, i)**2 + delta(y_col, i)**2)

    f9 = 0
    f10 = 0
    f11 = 0
    for i in range (2, df.shape[0], 1):
        # print(_delta(x_col, i) * _delta(x_col, i-1))
        # print(_delta(y_col, i) * _delta(y_col, i-1))
        # print(delta(x_col, i))
        # print(delta(x_col, i-1))
        # print(delta(y_col, i))
        # print(delta(y_col, i-1))
        # print()

        nominator = (-delta(y_col, i) * delta(x_col, i-1) + delta(y_col, i-1) * delta(x_col, i))
        denominator = (delta(x_col, i) * delta(x_col, i-1) + delta(y_col, i)   * delta(y_col, i-1))
        f9 += math.atan2(nominator, denominator)
        f10 += abs(f9)
        f11 += f9**2

    f12 = 0
    for i in range(1, df.shape[0], 1):
        if delta(t_col, i) == 0: continue
        f12 = max(f12, (delta(x_col, i)**2 + delta(y_col, i)**2) / delta(t_col, i)**2)

    f13 = t_col[-1:] - t_col[0]


In [26]:
for character in ascii_lowercase:
    for index in range (20):
        path = "data/letters-csv/" + character + "/" + character + "_" + str(index+1) + ".csv"
        df = pd.read_csv(path)
        

['data/letters-csv/',
 'data/letters-csv/r',
 'data/letters-csv/u',
 'data/letters-csv/i',
 'data/letters-csv/n',
 'data/letters-csv/g',
 'data/letters-csv/z',
 'data/letters-csv/t',
 'data/letters-csv/s',
 'data/letters-csv/a',
 'data/letters-csv/f',
 'data/letters-csv/o',
 'data/letters-csv/h',
 'data/letters-csv/m',
 'data/letters-csv/j',
 'data/letters-csv/c',
 'data/letters-csv/d',
 'data/letters-csv/v',
 'data/letters-csv/q',
 'data/letters-csv/x',
 'data/letters-csv/e',
 'data/letters-csv/b',
 'data/letters-csv/k',
 'data/letters-csv/l',
 'data/letters-csv/y',
 'data/letters-csv/p',
 'data/letters-csv/w']